# Initial Research stuff

In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
from scipy.stats import rankdata, stats
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from enum import Enum
import qgrid
from IPython.display import HTML

## Functions we need

In [2]:

def min_rank_changes(vec1, vec2, min_change=1):
    rk1 = rankdata(vec1)
    rk2 = rankdata(vec2)
    diff = np.abs(rk1-rk2)
    count = 0
    for val in diff:
        if val >= min_change:
            count+=1
    return count

# Load our data

In [3]:
df = pd.read_excel("initial-data.xlsx", sheet_name=1, index_col=0)
weights = pd.read_excel("initial-data.xlsx").iloc[0,2:]
orig_weights = deepcopy(weights)
orig_ranks = rankdata(-df["Value"])
del df["Value"]
df.index.name=None
criteria = weights.index.tolist()
alternatives = df.index.tolist()
display(weights)
display(df)

Category A     0.48775
Category B    0.115658
Category C    0.045709
Threat        0.071927
Population    0.278955
Name: 0, dtype: object

,Category A,Category B,Category C,,Population
Asset 23,0.667,0.0,0.000,0.767677,1.000000
Asset 19,0.667,0.0,0.000,0.696970,0.971020
Asset 15,0.667,0.4,0.000,0.070707,0.632661
Asset 25,0.500,0.0,0.000,0.080808,0.994852
Asset 26,0.667,0.4,0.000,0.707071,0.320068
Asset 14,0.667,0.4,0.000,0.545455,0.267750
Asset 22,0.667,0.0,0.000,0.010101,0.421111
Asset 21,0.500,0.0,0.000,0.393939,0.535068
Asset 12,0.667,0.4,0.000,0.313131,0.093636
Asset 3,0.667,0.0,0.000,0.939394,0.067646


In [4]:
step = 0.01
wg1 = widgets.FloatSlider(min=0, max=1, value=weights[0], step=step)
wg2 = widgets.FloatSlider(min=0, max=1, value=weights[1], step=step)
wg3 = widgets.FloatSlider(min=0, max=1, value=weights[2], step=step)
wg4 = widgets.FloatSlider(min=0, max=1, value=weights[3], step=step)
wg5 = widgets.FloatSlider(min=0, max=1, value=weights[4], step=step)
@interact(w1=wg1, w2=wg2, w3=wg3, w4=wg4, w5=wg5)
def calc_diff(w1, w2, w3, w4, w5):
    print(wg1)
    weights = pd.Series([w1, w2, w3, w4, w5])
    weights = np.array([weights]).transpose()
    new_scores = np.matmul(df.values, weights)
    orig_scores = np.matmul(df.values, orig_weights)
    new_rks = rankdata(new_scores)
    orig_rks = rankdata(orig_scores)
    diff_vals = new_rks - orig_rks
    display(diff_vals)
    display(orig_scores)
    display(new_scores)
    avg_diff = sum(np.abs(diff_vals))/len(new_scores)
    max_diff = max(np.abs(diff_vals))
    count2_change = min_rank_changes(orig_scores, new_scores, 2)
    count3_change = min_rank_changes(orig_scores, new_scores, 3)
    return {"average":avg_diff, "max":max_diff, 
            "projects changing at least 2 ranks":count2_change,
            "projects changing at least 3 ranks":count3_change,
           }

interactive(children=(FloatSlider(value=0.487750244926334, description='w1', max=1.0, step=0.01), FloatSlider(…

Find out how much I have to move each weight in order to induce
* A rank change
* At least 2 rank changes
* At least n rank changes
* Avg change at least ___
* Max change at least ____
* Count of at least _____ changes $\geq$ ___
* Maybe a matrix of the above for each criteria

# Algorithm to find first change

From good notes DL2022 page 18

## Code we need

In [5]:
def taxi_dist(r1, r2):
    '''Basic taxi cab metric'''
    diff = np.subtract(r1, r2)
    rval = np.sum(np.abs(diff))
    return rval

def count_diffs_dist(r1, r2, return_percent=False, min_dist=1):
    '''Counts the diffs between 2 rankings'''
    count = 0
    for v1,v2 in zip(r1,r2):
        if np.abs(v1-v2) >= min_dist:
            count += 1
    if return_percent:
        return count/len(r1)
    else:
        return count
    
def weights_diff(orig_wts, new_wts, do_percent=True, totaler=np.mean):
    '''Calculates the difference between 2 vectors, perhaps using percent diff.'''
    diff = np.abs(np.subtract(orig_wts,new_wts))
    if do_percent:
        for i in range(len(diff)):
            if orig_wts[i] != 0:
                diff = diff/np.abs(orig_wts[i])
    rval = totaler(diff)
    return rval

def family_wts(t, w0, pos):
    '''
    Changes the importance of the weight in w0 in position pos in the following fashion:
    when t=0 it is w0[pos], i.e. the initial value
    when t=1 its value is 1, i.e. the most important
    In between it scales linearly.
    And the remainder of values are scaled so that the total adds to one
    '''
    rval = deepcopy(w0)
    prev_sum = sum(w0)
    rval[pos] = (1-t)*w0[pos] + t*1
    rest_sum = prev_sum - w0[pos]
    new_sum = 1 - rval[pos]
    factor = new_sum / rest_sum
    for i in range(len(w0)):
        if i != pos:
            rval[i] *= factor
    return rval
    
def firstTime_old(A, w0, f, d, gamma, step=0.001, round_digits=None):
    '''My first attempt at the algorithm, ignore'''
    score0 = np.matmul(A,w0).tolist()
    if round_digits is not None:
        score0 = np.round(score0, round_digits)
    rank0 = rankdata(score0)
    t = 0 + step
    while t <= 1:
        w = f(t)
        score = np.matmul(A, w).tolist()
        if round_digits is not None:
            score = np.round(score, round_digits)
        #print(score)
        rank = rankdata(score)
        dist = d(rank0, rank)
        if dist >= gamma:
            return t
        t += step
    return None

def tie_locations(sorted_list, return_untied=True):
    '''
    Return list of locations with ties
    '''
    rval_tied = []
    rval_untied = [0]
    loc = 1
    while loc < len(sorted_list):
        if sorted_list[loc-1] != sorted_list[loc]:
            rval_untied.append(loc)
            loc += 1
        else:
            value = sorted_list[loc-1]
            # We want to remove every element that equals sorted_list[loc-1]
            for loc in range(loc, len(sorted_list)):
                if sorted_list[loc] != value:
                    break
                else:
                    rval_tied.append(loc)
    if return_untied:
        return rval_untied
    else:
        return rval_tied

def sort_and_remove_ties(vec, round_digits):
    vec_rnd = np.round(vec, round_digits)
    vec_rnd = np.sort(vec_rnd)
    untie_locs = tie_locations(vec_rnd)
    return vec_rnd[untie_locs]

def unique_pos(vec, round_digits):
    '''Returns the list of positions in the vector that are "unique", i.e. not duplicates.
    However the first "duplicate" does get returned.
    For [1,1,3,3,3,5,2,1,3,5] this function would return
    [0, 2, 5, 6]
    0=first instance of the number 1, it is not a duplicate, the rest are...at indices 1,7
    2=first instance of the number 3, it is not a duplicate, the rest are dupes at indices 3,4,8
    5=first instance of the number 5, not a dupe, but the 5 in the last index is a dupe
    6=first instance of the number 2, not a dupe, and there are no other 2's in the list
    '''
    vec = np.round(vec, round_digits)
    sort_pos = np.argsort(vec)
    rval = [sort_pos[0]]
    loc = 1
    while loc < len(sort_pos):
        if vec[sort_pos[loc-1]] != vec[sort_pos[loc]]:
            rval.append(sort_pos[loc])
            loc += 1
        else:
            value = vec[sort_pos[loc-1]]
            for loc in range(loc, len(vec)):
                if vec[sort_pos[loc]] != value:
                    break
    return rval
        
def firstTime2(A, w0, f, d, gamma, step=0.001, ignore_ties_ndigits=None):
    '''My second attempt, where I ignore close ties, but that had failings'''
    score0 = np.array(np.matmul(A,w0).tolist())
    uniques = None
    if ignore_ties_ndigits is not None:
        #print("Rounding")
        uniques = unique_pos(score0, ignore_ties_ndigits)
        score0 = score0[uniques]
    #print(score0)
    rank0 = rankdata(score0)
    t = 0 + step
    while t <= 1:
        w = f(t)
        score = np.array(np.matmul(A, w).tolist())
        if ignore_ties_ndigits is not None:
            score = score[uniques]
        rank = rankdata(score)
        dist = d(rank0, rank)
        if dist >= gamma:
            return t
        t += step
    return None

class WeightDiffsCalc(Enum):
    PARAM = 1
    COMBINE = 2
    LOC_CHANGED = 3

def firstTime(A, w0, f, d, gamma, step=0.001, wt_diffs_calc=WeightDiffsCalc.PARAM, wt_combine_loc = None,
              wt_changes_percent=True, wt_changes_totaler=np.mean, return_all_data=False):
    '''The actual algorithm to find the first time a ranking changes due to a parameter change'''
    score0 = np.array(np.matmul(A,w0).tolist())
    rank0 = rankdata(-score0)
    t = 0 + step
    while t <= 1:
        w = f(t)
        score = np.array(np.matmul(A, w).tolist())
        rank = rankdata(-score)
        dist = d(rank0, rank)
        if dist >= gamma:
            weight_changes_total = weights_diff(w0, w, do_percent=wt_changes_percent, totaler=wt_changes_totaler)
            if wt_diffs_calc is WeightDiffsCalc.COMBINE:
                # We need to see the new weight changes from original
                rval = weight_changes_total
            elif wt_diffs_calc is WeightDiffsCalc.LOC_CHANGED:
                weight_changes_local = weights_diff(w0[wt_combine_loc:(wt_combine_loc+1)], w[wt_combine_loc:(wt_combine_loc+1)],
                                                    do_percent=wt_changes_percent, totaler=wt_changes_totaler)
                rval = weight_changes_local
            else:
                rval = t
            if return_all_data:
                return {
                    "rval": rval,
                    "param": t,
                    "weight_changes_total": weight_changes_total,
                    "weight_changes_local": weight_changes_local,
                    "weights": w,
                    "scores": score,
                    "ranks": rank
                }
            else:
                return rval
        t += step
    return None


## Testing functions quickly

In [6]:
# Testing the function
family_wts(0.5, [0.5, 0.4, 0.1], 0)

[0.75, 0.2, 0.05]

In [7]:
# Testing
tie_locations([1, 1, 1, 2, 3, 4, 4, 5], return_untied=True)

[0, 3, 4, 5, 7]

In [8]:
# Testing unique_pos function
a = np.array([3, 2, 3, 4, 5, 3, 3, 1, 7, 1])
index = unique_pos(a, 1)
print(index)
print(a)
print(a[index])

[7, 1, 0, 3, 4, 8]
[3 2 3 4 5 3 3 1 7 1]
[1 2 3 4 5 7]


## Trying out our algorithm

### First attempt
Quite crude

In [9]:
A = df.values
w0 = weights
round_digits = 1
for min_dist in [1,2,3,4,5,6,7]:
    for i in range(len(w0)):
        f = lambda t : family_wts(t, w0, i)
        rval = firstTime(A, w0, f, taxi_dist, min_dist)
        print("RankChange >= "+str(min_dist)+" for Weight["+str(i)+"]="+str(rval))
    print()

RankChange >= 1 for Weight[0]=0.008
RankChange >= 1 for Weight[1]=0.004
RankChange >= 1 for Weight[2]=0.026000000000000016
RankChange >= 1 for Weight[3]=0.001
RankChange >= 1 for Weight[4]=0.001

RankChange >= 2 for Weight[0]=0.008
RankChange >= 2 for Weight[1]=0.004
RankChange >= 2 for Weight[2]=0.026000000000000016
RankChange >= 2 for Weight[3]=0.001
RankChange >= 2 for Weight[4]=0.001

RankChange >= 3 for Weight[0]=0.008
RankChange >= 3 for Weight[1]=0.03800000000000003
RankChange >= 3 for Weight[2]=0.05300000000000004
RankChange >= 3 for Weight[3]=0.004
RankChange >= 3 for Weight[4]=0.001

RankChange >= 4 for Weight[0]=0.008
RankChange >= 4 for Weight[1]=0.03800000000000003
RankChange >= 4 for Weight[2]=0.05300000000000004
RankChange >= 4 for Weight[3]=0.004
RankChange >= 4 for Weight[4]=0.001

RankChange >= 5 for Weight[0]=0.027000000000000017
RankChange >= 5 for Weight[1]=0.05600000000000004
RankChange >= 5 for Weight[2]=0.11300000000000009
RankChange >= 5 for Weight[3]=0.0140000

### Second attempt
Still pretty crude

In [10]:
A = df.values
w0 = weights
round_digits = 3
for min_dist in [1,2,3,4,5,6,7]:
    for i in range(len(w0)):
        f = lambda t : family_wts(t, w0, i)
        rval = firstTime(A, w0, f, count_diffs_dist, min_dist,)
        print("CountDiffs >= "+str(min_dist)+" for Weight["+str(i)+"]="+str(rval))
    print()

CountDiffs >= 1 for Weight[0]=0.008
CountDiffs >= 1 for Weight[1]=0.004
CountDiffs >= 1 for Weight[2]=0.026000000000000016
CountDiffs >= 1 for Weight[3]=0.001
CountDiffs >= 1 for Weight[4]=0.001

CountDiffs >= 2 for Weight[0]=0.008
CountDiffs >= 2 for Weight[1]=0.004
CountDiffs >= 2 for Weight[2]=0.026000000000000016
CountDiffs >= 2 for Weight[3]=0.001
CountDiffs >= 2 for Weight[4]=0.001

CountDiffs >= 3 for Weight[0]=0.008
CountDiffs >= 3 for Weight[1]=0.03800000000000003
CountDiffs >= 3 for Weight[2]=0.05300000000000004
CountDiffs >= 3 for Weight[3]=0.004
CountDiffs >= 3 for Weight[4]=0.001

CountDiffs >= 4 for Weight[0]=0.008
CountDiffs >= 4 for Weight[1]=0.03800000000000003
CountDiffs >= 4 for Weight[2]=0.11300000000000009
CountDiffs >= 4 for Weight[3]=0.004
CountDiffs >= 4 for Weight[4]=0.001

CountDiffs >= 5 for Weight[0]=0.027000000000000017
CountDiffs >= 5 for Weight[1]=0.05600000000000004
CountDiffs >= 5 for Weight[2]=0.11600000000000009
CountDiffs >= 5 for Weight[3]=0.0140000

### A little less crude
But a lot of information

In [11]:
A = df.values
w0 = weights
for count_dist in [1,2,3,4,5,6,7]:
    for min_dist in [1,2,3,4,10]: 
        for i in range(len(w0)):
            f = lambda t : family_wts(t, w0, i)
            d = lambda r1, r2 : count_diffs_dist(r1, r2, min_dist=min_dist)
            rval = firstTime(A, w0, f, d, count_dist)
            print("To get at least "+str(count_dist)+" projects to change rank at least " + str(min_dist)+" for Weight["+str(i)+"]="+str(rval))
        print()
    print()
    print()

To get at least 1 projects to change rank at least 1 for Weight[0]=0.008
To get at least 1 projects to change rank at least 1 for Weight[1]=0.004
To get at least 1 projects to change rank at least 1 for Weight[2]=0.026000000000000016
To get at least 1 projects to change rank at least 1 for Weight[3]=0.001
To get at least 1 projects to change rank at least 1 for Weight[4]=0.001

To get at least 1 projects to change rank at least 2 for Weight[0]=0.036000000000000025
To get at least 1 projects to change rank at least 2 for Weight[1]=0.05600000000000004
To get at least 1 projects to change rank at least 2 for Weight[2]=0.05300000000000004
To get at least 1 projects to change rank at least 2 for Weight[3]=0.015000000000000006
To get at least 1 projects to change rank at least 2 for Weight[4]=0.013000000000000005

To get at least 1 projects to change rank at least 3 for Weight[0]=0.09100000000000007
To get at least 1 projects to change rank at least 3 for Weight[1]=0.17900000000000013
To get

To get at least 4 projects to change rank at least 10 for Weight[4]=0.7510000000000006



To get at least 5 projects to change rank at least 1 for Weight[0]=0.027000000000000017
To get at least 5 projects to change rank at least 1 for Weight[1]=0.05600000000000004
To get at least 5 projects to change rank at least 1 for Weight[2]=0.11600000000000009
To get at least 5 projects to change rank at least 1 for Weight[3]=0.014000000000000005
To get at least 5 projects to change rank at least 1 for Weight[4]=0.002

To get at least 5 projects to change rank at least 2 for Weight[0]=0.10300000000000008
To get at least 5 projects to change rank at least 2 for Weight[1]=0.21100000000000016
To get at least 5 projects to change rank at least 2 for Weight[2]=None
To get at least 5 projects to change rank at least 2 for Weight[3]=0.023000000000000013
To get at least 5 projects to change rank at least 2 for Weight[4]=0.027000000000000017

To get at least 5 projects to change rank at least 3 for Weight

In [12]:
A = df.values
w0 = weights
for count_dist in [1,2,3,4,5]:
    for min_dist in [1,2,3]: 
        for i in range(len(w0)):
            f = lambda t : family_wts(t, w0, i)
            d = lambda r1, r2 : count_diffs_dist(r1, r2, min_dist=min_dist)
            rval = firstTime(A, w0, f, d, count_dist,
                            wt_diffs_calc=WeightDiffsCalc.LOC_CHANGED,
                            wt_combine_loc=i,
                            wt_changes_percent=True,
                            wt_changes_totaler=np.mean)
            if rval is None:
                print("To get at least "+str(count_dist)+" projects to change rank at least " + str(min_dist)+" for Weight["+str(i)+"]=Impossible")
            else:
                print("To get at least "+str(count_dist)+" projects to change rank at least " + str(min_dist)+" for Weight["+str(i)+"]={:.2%}".format(rval))
        print()
    print()
    print()

To get at least 1 projects to change rank at least 1 for Weight[0]=0.84%
To get at least 1 projects to change rank at least 1 for Weight[1]=3.06%
To get at least 1 projects to change rank at least 1 for Weight[2]=54.28%
To get at least 1 projects to change rank at least 1 for Weight[3]=1.29%
To get at least 1 projects to change rank at least 1 for Weight[4]=0.26%

To get at least 1 projects to change rank at least 2 for Weight[0]=3.78%
To get at least 1 projects to change rank at least 2 for Weight[1]=42.82%
To get at least 1 projects to change rank at least 2 for Weight[2]=110.65%
To get at least 1 projects to change rank at least 2 for Weight[3]=19.35%
To get at least 1 projects to change rank at least 2 for Weight[4]=3.36%

To get at least 1 projects to change rank at least 3 for Weight[0]=9.56%
To get at least 1 projects to change rank at least 3 for Weight[1]=136.87%
To get at least 1 projects to change rank at least 3 for Weight[2]=235.91%
To get at least 1 projects to change ran

In [13]:
A = df.values
w0 = weights
for count_dist in [1,2,3,4,5,6,7]:
    for min_dist in [1,2,3,4]:
        infos = []
        for i in range(len(w0)):
            f = lambda t : family_wts(t, w0, i)
            d = lambda r1, r2 : count_diffs_dist(r1, r2, min_dist=min_dist)
            rval = firstTime(A, w0, f, d, count_dist,
                            wt_diffs_calc=WeightDiffsCalc.LOC_CHANGED,
                            wt_combine_loc=i,
                            wt_changes_percent=True,
                            wt_changes_totaler=np.mean)
            if rval is not None:
                infos.append(rval)
        if len(infos) > 0:
            quickest = np.min(infos)
        else:
            quickest = None
        print("Smallest change to get at least "+str(count_dist)+" projects to change rank at least " + str(min_dist)+" requires weights change {:0.2%}".format(rval))
    print()
    print()

Smallest change to get at least 1 projects to change rank at least 1 requires weights change 0.26%
Smallest change to get at least 1 projects to change rank at least 2 requires weights change 3.36%
Smallest change to get at least 1 projects to change rank at least 3 requires weights change 16.54%
Smallest change to get at least 1 projects to change rank at least 4 requires weights change 18.09%


Smallest change to get at least 2 projects to change rank at least 1 requires weights change 0.26%
Smallest change to get at least 2 projects to change rank at least 2 requires weights change 3.88%
Smallest change to get at least 2 projects to change rank at least 3 requires weights change 17.58%
Smallest change to get at least 2 projects to change rank at least 4 requires weights change 19.13%


Smallest change to get at least 3 projects to change rank at least 1 requires weights change 0.26%
Smallest change to get at least 3 projects to change rank at least 2 requires weights change 4.39%
Sm

### Actual version demo'd

In [14]:
# Actual calculation!
nalts_list = [1,2,3,4,5,6,7]
nrk_changes_list = [1,2,3,4,5]
df_columns = ["{:d} Alts".format(i) for i in nalts_list]
df_index = ["{:d} Ranks".format(i) for i in nrk_changes_list]
rval_df = pd.DataFrame(columns=df_columns,
                      index=df_index)
weights_df = pd.DataFrame(columns=df_columns, index=df_index, dtype=object)
new_ranks_df = pd.DataFrame(columns=df_columns, index=df_index, dtype=object)
A = df.values
w0 = orig_weights
for count_dist in nalts_list:
    for min_dist in nrk_changes_list:
        infos = []
        aweights = []
        ascores = []
        aranks = []
        for i in range(len(w0)):
            f = lambda t : family_wts(t, w0, i)
            d = lambda r1, r2 : count_diffs_dist(r1, r2, min_dist=min_dist)
            rval_info2 = firstTime(A, w0, f, d, count_dist,
                                  wt_diffs_calc=WeightDiffsCalc.LOC_CHANGED,
                                  wt_combine_loc=i,
                                  wt_changes_percent=True,
                                  wt_changes_totaler=np.mean,
                                 return_all_data=True)
            #display(rval_info)
            if rval_info2 is not None:
                rval = rval_info2['rval']
                infos.append(rval)
                aweights.append(rval_info2["weights"])
                ascores.append(rval_info2["scores"])
                aranks.append(rval_info2["ranks"])
        if len(infos) > 0:
            min_index = np.argmin(infos)
            quickest = infos[min_index]
            weights_df["{:d} Alts".format(count_dist)]["{:d} Ranks".format(min_dist)] = aweights[min_index]
            new_ranks_df["{:d} Alts".format(count_dist)]["{:d} Ranks".format(min_dist)] = aranks[min_index]
        else:
            quickest = None
        
        rval_df["{:d} Alts".format(count_dist)]["{:d} Ranks".format(min_dist)] = quickest
        print("Smallest change to get at least "+str(count_dist)+" projects to change rank at least " + str(min_dist)+" requires weights change {:0.2%}".format(rval))
    print()
    print()
    


Smallest change to get at least 1 projects to change rank at least 1 requires weights change 0.26%
Smallest change to get at least 1 projects to change rank at least 2 requires weights change 3.36%
Smallest change to get at least 1 projects to change rank at least 3 requires weights change 16.54%
Smallest change to get at least 1 projects to change rank at least 4 requires weights change 18.09%
Smallest change to get at least 1 projects to change rank at least 5 requires weights change 19.90%


Smallest change to get at least 2 projects to change rank at least 1 requires weights change 0.26%
Smallest change to get at least 2 projects to change rank at least 2 requires weights change 3.88%
Smallest change to get at least 2 projects to change rank at least 3 requires weights change 17.58%
Smallest change to get at least 2 projects to change rank at least 4 requires weights change 19.13%
Smallest change to get at least 2 projects to change rank at least 5 requires weights change 38.51%




Some styling to make my 2 tables show up side by side

In [29]:
%%HTML
<style>
.flex-container {
    display: flex;
    flex-flox: row wrap;
}
</style>

Now make the UI to do this calculation

In [26]:
display(qgrid.show_grid(rval_df))

row_choice = widgets.Dropdown(
    options = df_index,
    description = "Row"
)
col_choice = widgets.Dropdown(
    options = df_columns,
    description = "Column"
)
@interact(col=col_choice, row=row_choice)
def show_it(col,row):
    rank_report_df = pd.DataFrame({
        "Original Ranks":orig_ranks, 
        "New Ranks":new_ranks_df[col][row],
        "Diff": orig_ranks - new_ranks_df[col][row],
    }, index=df.index)
    div1="<div><H2>Weights Information</H2>"+pd.DataFrame({"Original Weights":orig_weights, "New Weights": weights_df[col][row]}).to_html()+"</div>"
    div2="<div style=\"padding-left:40px;\"><h2>Rank Information</h2>"+rank_report_df.to_html()+"</div>"
    div1a="<div> </div>"
    total_div='<div class="flex-container">'+div1+div1a+div2+"</div>"
    display(HTML(total_div))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

interactive(children=(Dropdown(description='Column', options=('1 Alts', '2 Alts', '3 Alts', '4 Alts', '5 Alts'…